# Assignment 6

### Part A

In [1362]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [1363]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [1364]:
def exp_return(data, P):
    # exp_return(data, 40) should output 0.07500000000000002

    length = len(data["Price in 1 Year"]) 

    # Array storing returns
    R = []
    for i in range(length):
        R.append((data["Price in 1 Year"][i] / P) - 1) 

    # Find expected return
    expected_return = 0
    for i in range(length):
        expected_return += R[i] * data["Probability"][i]  

    return expected_return

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [1366]:
def volatility(data, P):
    # volatility(data,40) should output 0.21301408404140795

    length = len(data["Price in 1 Year"]) 

    # Array storing returns
    R = []
    for i in range(length):
        R.append((data["Price in 1 Year"][i] / P) - 1) 
    

    #Finding variance
    variance = 0
    expected_return = exp_return(data, P)
    for i in range(length):
        variance += (R[i] - expected_return)**2 * data["Probability"][i]  

    # Return standardard deviation aka volatility
    return variance**0.5

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [1368]:
def interval(data, P):
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]

    confidence_intervals = []

    standard_error = volatility(data, P) / len(data["Price in 1 Year"])**0.5

    confidence_intervals.append( exp_return(data, P) - 1.96 * standard_error )
    confidence_intervals.append( exp_return(data, P) + 1.96 * standard_error )


    

    return confidence_intervals

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [1370]:
def exp_return_capm(beta, market_return, rf):
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028

    return rf + beta * (market_return - rf)

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [1372]:
def clean_data(path):
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)


    # Make a new column that uses the to_numeric() function -> everything that is not numeric is set to NaN
    # Then all NaN's in that column are dropped
    df["Numeric"] = pd.to_numeric(df["RET"], errors ='coerce') 
    df = df.dropna(subset=['Numeric'])
    df = df.drop('Numeric', axis=1)

    # Create a Series containing the counts of occurences 
    # If there are over 500 occurences then i add them to an array, then create the final filtered dataframe with the values of that array
    value_counts = df["PERMNO"].value_counts()
    securites_with_over_500 = []
    for index,value in value_counts.items():
        if (value >= 500):
            securites_with_over_500.append(index)

    df = df[df['PERMNO'].isin(securites_with_over_500)]

    
    
    return df


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [1374]:
def avg_return(data):
    # This should yield data with 679 rows × 2 columns

    
    output = pd.DataFrame(data = {
        "CUSIP" : [],
        "Average Return" : []
    })

    # Create a Series containing the counts of occurences 
    df = clean_data(data)
    value_counts = df["CUSIP"].value_counts()

    for index,value in value_counts.items():

        # Strategy used: Create a new filtered dataframe with only the current item in CUSIP, repeat for each item
        average_return = 0
        filtered_df = df[df['CUSIP'].isin([index])].copy()
        filtered_df["Numeric"] = pd.to_numeric(filtered_df["RET"], errors ='coerce') 
        average_return += filtered_df['Numeric'].sum()
        average_return = average_return / value

        #Concatenate to output
        data = pd.DataFrame(data = {
        "CUSIP" : [index],
        "Average Return" : [average_return]
        })
        output = pd.concat([output, data])

    # Create index
    output = output.reset_index(drop=True)


    
    
    return output

In [1376]:
def security_volatilities(data):
    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : [],
        "Volatility" : []
    })


    # Create a Series containing the counts of occurences 
    df = clean_data(data)
    value_counts = df["CUSIP"].value_counts()

    for index,value in value_counts.items():

        #Find the average return (same as above since this doesn't take much time)
        average_return = 0
        filtered_df = df[df['CUSIP'].isin([index])].copy()
        filtered_df["Numeric"] = pd.to_numeric(filtered_df["RET"], errors ='coerce') 
        average_return += filtered_df['Numeric'].sum()
        average_return = average_return / value

        # Find the variance
        variance = 0
        for i in range( value ):
            variance += (filtered_df["Numeric"].iloc[i] - average_return)**2
        variance = variance / (value - 1)

        # Finding volatility
        volatility = variance**0.5


        #Concatenate to output
        data = pd.DataFrame(data = {
        "CUSIP" : [index],
        "Volatility" : [volatility]
        })
        output = pd.concat([output, data])
    

    output = output.reset_index(drop=True)


    
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [1378]:
def correlation(ret_data, vol_data):
    # correlation(rets, vols) should yield 0.44837090727059614

    both_dfs = pd.concat([ret_data, vol_data], axis=1)

    corr = both_dfs["Average Return"].corr(both_dfs["Volatility"])

    # SHOULD TAKE MAX 30 SECONDS TO RUN (takes me 20secs)
    
    return corr